In [42]:
import random, struct,math
import argparse

In [43]:
def int2bin(iIn,iBW):
    iBW = iBW + 1
    if iIn >= 0:
        bOut = bin(iIn).replace('0b','').rjust(iBW,'0')
    else :
        bOut = bin(iIn & (pow(2,iBW)-1)).replace('0b','').rjust(iBW,'1')
    return bOut[1:]

In [44]:
def XOR(iA,iB):
    if iA != iB :
        iOut = '1'
    else : 
        iOut = '0'
    return iOut

In [45]:
parser = argparse.ArgumentParser(description='fixed_mac')
parser.add_argument('--full_bits', type=int, default=8, help='Number of Quantization Bits')
parser.add_argument('--frac_bits', type=int, default=6, help='Number of Quantization Bits')
parser.add_argument('--bBW', type=int, default=2, help='Number of bit width')
args = parser.parse_args(args=[])

In [52]:
class	fxp:
	def	__init__(self, bIn, iBWF):
		self.iFullBW	= len(bIn)
		self.iIntgBW	= self.iFullBW - iBWF
		self.bSign		= bIn[0]
		self.bIntg		= bIn[:self.iIntgBW]
		self.bFrac		= bIn[self.iIntgBW:]
		self.fFull		= 0
		try:
			for idx, bit in enumerate(bIn):
				if	idx == 0:
					self.fFull = self.fFull + int(bit,2) * -pow(2, self.iIntgBW - 1)
				else:
					self.fFull = self.fFull + int(bit,2) * pow(2, self.iIntgBW - 1 - idx)
		except:
			print(bIn)
		self.dispFull	= self.bIntg +"."+ self.bFrac 
		return

In [53]:
class flp2fix:
    def __init__(self, fIn, iBW, iBWF):
        self.fMin = - 2 ** (iBW - iBWF - 1)
        self.fMax = (2 ** (iBW-1) - 1) * (2 ** -iBWF)
        self.fResol		= 2 ** -iBWF
        if fIn < self.fMin or fIn > self.fMax:
            print(f'({fIn}): Out of input range ({self.fMax}/{self.fMin}) during flp -> fix converting ')
        self.iBW = iBW
        self.iBWI = iBW - iBWF
        self.iBWF = iBWF

		self.iFLP2INT	= abs(int(fIn * 2 ** iBWF))
		if fIn < 0:
			self.iFLP2INT = 2 ** (iBW-1) - self.iFLP2INT

		if fIn >= 0:
			self.bFull = bin(self.iFLP2INT)[2:].rjust(iBW, '0')
		else:
			self.bFull = '1'+bin(self.iFLP2INT)[2:].rjust(iBW-1, '0')
			if len(self.bFull) > iBW:
				self.bFull = '0' * iBW

		self.cssFxp		= fxp(self.bFull, self.iBWF)
		self.bSign		= self.cssFxp.bSign
		self.bIntg		= self.cssFxp.bIntg
		self.bFrac		= self.cssFxp.bFrac
		self.fFull		= self.cssFxp.fFull
		return

In [54]:
a = flp2fix(0.3,args.full_bits,args.frac_bits).fFull

In [55]:
print(a)

0.296875


In [60]:
b = flp2fix(0.3,args.full_bits,args.frac_bits).bFull

In [61]:
print(b)

00010011


In [71]:
for i in range(10) :
    a = random.random()
    b = flp2fix(a,args.full_bits,args.frac_bits).fFull
    c = flp2fix(a,args.full_bits,args.frac_bits).bFull
    d = flp2fix(b,args.full_bits,args.frac_bits).fFull
    print(f'random number is : {a}')
    print(f'changed fixed number is : {b}')
    print(f'binary fixed number is : {c[2:]}')
    print(f'--------------------------------------------')

random number is : 0.28047818148533676
changed fixed number is : 0.265625
binary fixed number is : 010001
--------------------------------------------
random number is : 0.24747249720567366
changed fixed number is : 0.234375
binary fixed number is : 001111
--------------------------------------------
random number is : 0.04707222486282936
changed fixed number is : 0.046875
binary fixed number is : 000011
--------------------------------------------
random number is : 0.056665799891180924
changed fixed number is : 0.046875
binary fixed number is : 000011
--------------------------------------------
random number is : 0.4908722892525552
changed fixed number is : 0.484375
binary fixed number is : 011111
--------------------------------------------
random number is : 0.5019707026435921
changed fixed number is : 0.5
binary fixed number is : 100000
--------------------------------------------
random number is : 0.5662940754777158
changed fixed number is : 0.5625
binary fixed number is : 1001